In [ ]:
!pip install fairseq2
!pip install pydub sentencepiece 
!pip install git+https://github.com/facebookresearch/seamless_communication.git

In [ ]:
import io
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import mmap
import numpy
import soundfile
import torchaudio
import torch

from collections import defaultdict
from IPython.display import Audio, display
from pathlib import Path
from pydub import AudioSegment

from seamless_communication.inference import Translator
from seamless_communication.streaming.dataloaders.s2tt import SileroVADSilenceRemover

In [ ]:
!wget "https://d11ywzt2xtszji.cloudfront.net/SeamlessExpressive.tar.gz?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiZ2w1dG9obDh0ODN4Ym91NDViYzh3amxsIiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZDExeXd6dDJ4dHN6amkuY2xvdWRmcm9udC5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NTg3Nzk0MTd9fX1dfQ__&Signature=bWDs-HXUQxMg1hMThnSJ0cQeGgNF7KBu5arLKife0k8nGkA5sL6mfFMp9fezXTnE3TdB2o326To7SrRb1mUIFeTp1c3hvVrBhM9KIhThDOx55wZNdC6NCbktV5nhQt5qoWgNURsOqlNSqgZwrXhvFUfB-tcmm53JVnvipWC5qqRKdC95csVUE-mhEKp0khjY0OHtIhdIPv2aLCgPicvsDAHCb5s%7EKQ9wwx5OEoI02S6cXmhm%7EgrPy2nJcaoMDerGYZh0u0PnfmgCWhQmBAoyybPvFyUOLU2osLpzInXjW-gJf1KO-9kQiNq2mg96b6Jvr5Qi3pZBgTBLA2nV-AJiZQ__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=823543226689486" -O /content/SeamlessExpressive.tar.gz

!tar -xzvf /content/SeamlessExpressive.tar.gz


In [ ]:
import os

# Input and output directories
vad_dir = "/kaggle/input/sample-video/EY_VAD"
output_dir = "/kaggle/working/translated"
os.makedirs(output_dir, exist_ok=True)

# Target language
tgt_lang = "spa"

# Loop through all VAD segments
for file in sorted(os.listdir(vad_dir)):
    if file.endswith(".wav"):
        in_file = os.path.join(vad_dir, file)
        out_file = os.path.join(output_dir, file)  # same name as input

        print(f"🔹 Translating {file} -> {out_file}")

        # Run Seamless Expressive translation
        !expressivity_predict {in_file} --tgt_lang {tgt_lang} \
            --model_name seamless_expressivity --vocoder_name vocoder_pretssel \
            --gated-model-dir SeamlessExpressive --output_path {out_file}

print("\n✅ All translated files saved in:", output_dir)


In [ ]:
from IPython.display import Audio, display
import os

# 🔹 Paths to original and translated segment folders
orig_folder = "/kaggle/input/sample-video/EY_VAD"  # your original segments
trans_folder = "/kaggle/working/translated"  # translated segments

# 🔹 Get all segment files sorted
orig_files = sorted([f for f in os.listdir(orig_folder) if f.endswith(".wav")])
trans_files = sorted([f for f in os.listdir(trans_folder) if f.endswith(".wav")])

# 🔹 Play each segment pair
for i, (orig_f, trans_f) in enumerate(zip(orig_files, trans_files)):
    print(f"\nSegment {i}: {orig_f} (Original)")
    display(Audio(os.path.join(orig_folder, orig_f), autoplay=False, normalize=True, rate=16000))
    
    print(f"Segment {i}: {trans_f} (Translated)")
    display(Audio(os.path.join(trans_folder, trans_f), autoplay=False, normalize=True, rate=16000))


In [ ]:
# import shutil

# # Path to your VAD folder
# source_dir = "/kaggle/working/translated1"
# output_zip = "/kaggle/working/EY_translated"

# # Create zip of only VAD directory
# shutil.make_archive(output_zip.replace(".zip", ""), 'zip', source_dir)

# print(f"Zipped file saved at: {output_zip}")


In [ ]:
!pip install torch torchaudio numpy

In [ ]:
# !pip install openvoice-cli

In [ ]:
# !openvoice_cli --help

In [ ]:
# import sys
# !{sys.executable} -m pip show openvoice-cli

In [ ]:
# !python -m openvoice_cli --help


In [ ]:
# import os
# import json
# import math
# import shlex
# import subprocess
# from collections import defaultdict
# from pathlib import Path

# import torch
# import torchaudio
# import numpy as np

# # -------- CONFIG --------
# original_vad_dir = "/kaggle/working/translated"            # original VAD segments
# diarization_json = "/kaggle/input/sample-video/EY_VAD/speaker_labels.json"  # pyannote output
# translated_dir = "/kaggle/input/sample-video/translated_segments"   # translated segments (read-only)
# speaker_refs_dir = "/kaggle/working/speaker_refs"             # will be created
# cloned_dir = "/kaggle/working/cloned"                         # final outputs
# padded_dir = "/kaggle/working/translated_padded"              # temp padded files

# ref_seconds = 6.0          # how many seconds of reference audio to build per speaker
# target_sr = 22050          # OpenVoice-friendly sample rate
# device = "cuda" if torch.cuda.is_available() else "cpu"
# openvoice_cli_cmd = "python -m openvoice_cli single"  # CLI entry

# # ------------------------

# # Create folders
# os.makedirs(speaker_refs_dir, exist_ok=True)
# os.makedirs(cloned_dir, exist_ok=True)
# os.makedirs(padded_dir, exist_ok=True)

# # --------- Helpers ---------
# def load_mono_np(path):
#     wav, sr = torchaudio.load(path)
#     if wav.shape[0] > 1:
#         wav = wav.mean(dim=0, keepdim=True)
#     return wav.squeeze(0).cpu().numpy(), sr

# def resample_np(arr, orig_sr, new_sr):
#     if orig_sr == new_sr:
#         return arr
#     t = torch.from_numpy(arr).unsqueeze(0)
#     resampler = torchaudio.transforms.Resample(orig_sr, new_sr)
#     t2 = resampler(t)
#     return t2.squeeze(0).cpu().numpy()

# # Load diarization JSON
# with open(diarization_json, "r") as f:
#     diar = json.load(f)

# # Build speaker -> occurrences mapping
# speaker_occ = defaultdict(list)
# for entry in diar:
#     segfile = entry.get("segment_file")
#     for sp in entry.get("speakers", []):
#         speaker_occ[sp["speaker"]].append((segfile, float(sp["start"]), float(sp["end"])))

# print(f"[INFO] Found {len(speaker_occ)} speakers.")

# # -------- Build speaker references ---------
# for speaker, occs in speaker_occ.items():
#     parts = []
#     accum_sec = 0.0
#     for segfile, start, end in occs:
#         seg_path = os.path.join(original_vad_dir, segfile)
#         if not os.path.exists(seg_path):
#             print(f"[WARN] missing {seg_path} — skipping")
#             continue
#         arr, sr = load_mono_np(seg_path)
#         s_idx = int(math.floor(start * sr))
#         e_idx = int(math.ceil(end * sr))
#         clip = arr[s_idx:e_idx]
#         if clip.size == 0:
#             continue
#         if sr != target_sr:
#             clip = resample_np(clip, sr, target_sr)
#             sr = target_sr
#         parts.append(clip)
#         accum_sec += clip.shape[0] / float(target_sr)
#         if accum_sec >= ref_seconds:
#             break

#     if len(parts) == 0:
#         print(f"[WARN] No reference for {speaker} — skipping.")
#         continue

#     ref_audio = np.concatenate(parts, axis=0)
#     max_samples = int(ref_seconds * target_sr)
#     if ref_audio.shape[0] > max_samples:
#         ref_audio = ref_audio[:max_samples]
#     elif ref_audio.shape[0] < max_samples:
#         pad = np.zeros(max_samples - ref_audio.shape[0], dtype=ref_audio.dtype)
#         ref_audio = np.concatenate([ref_audio, pad], axis=0)

#     # normalize
#     peak = np.max(np.abs(ref_audio)) + 1e-9
#     ref_audio = (ref_audio / peak) * 0.95

#     # save
#     ref_path = os.path.join(speaker_refs_dir, f"{speaker}.wav")
#     tensor = torch.from_numpy(ref_audio).unsqueeze(0)
#     torchaudio.save(ref_path, tensor, sample_rate=target_sr)
#     print(f"[OK] Saved ref for {speaker} -> {ref_path}")

# print(f"[INFO] Speaker references done at: {speaker_refs_dir}")

# # -------- Primary speaker helper ---------
# def primary_speaker_for_segment(segfile):
#     for entry in diar:
#         if entry.get("segment_file") == segfile:
#             if not entry.get("speakers"):
#                 return None
#             durations = {sp["speaker"]: float(sp["end"])-float(sp["start"]) for sp in entry["speakers"]}
#             return max(durations, key=durations.get)
#     return None

# # --------- Cloning ---------
# translated_files = sorted([f for f in os.listdir(translated_dir) if f.endswith(".wav")])
# for fname in translated_files:
#     in_path_orig = os.path.join(translated_dir, fname)
#     speaker = primary_speaker_for_segment(fname)
#     if speaker is None:
#         print(f"[SKIP] No diarization for {fname}")
#         continue
#     ref_path = os.path.join(speaker_refs_dir, f"{speaker}.wav")
#     if not os.path.exists(ref_path):
#         print(f"[SKIP] Missing reference for speaker {speaker}")
#         continue

#     # Load and pad if too short
#     audio, sr = load_mono_np(in_path_orig)
#     min_len = int(ref_seconds * target_sr * 0.2)  # arbitrary minimum for OpenVoice
#     if audio.shape[0] < min_len:
#         pad = np.zeros(min_len - audio.shape[0], dtype=audio.dtype)
#         audio = np.concatenate([audio, pad], axis=0)

#     tmp_path = os.path.join(padded_dir, fname)
#     torchaudio.save(tmp_path, torch.from_numpy(audio).unsqueeze(0), sample_rate=sr)

#     out_path = os.path.join(cloned_dir, fname)
#     cmd = f"{openvoice_cli_cmd} -i {shlex.quote(tmp_path)} -r {shlex.quote(ref_path)} -o {shlex.quote(out_path)} -d {device}"
#     print(f"[RUN] Cloning {fname} -> speaker {speaker}")
#     try:
#         subprocess.run(cmd, shell=True, check=True)
#     except subprocess.CalledProcessError as e:
#         print(f"[ERR] OpenVoice failed for {fname} (err {e.returncode})")
#         continue

# print(f"[INFO] All done. Cloned files are in {cloned_dir}")


In [ ]:
# import os
# import json
# import math
# import shlex
# import subprocess
# from collections import defaultdict

# import torch
# import torchaudio
# import numpy as np

# # -------- CONFIG --------
# original_vad_dir = "/kaggle/input/sample-video/vad"           
# diarization_json = "/kaggle/input/sample-video/vad/speaker_labels.json"
# translated_dir = "/kaggle/input/sample-video/translated_segments"
# speaker_refs_dir = "/kaggle/working/speaker_reference"
# cloned_dir = "/kaggle/working/cloned_audio"
# padded_dir = "/kaggle/working/translated_padded"

# ref_seconds = 6.0          # seconds of reference audio per speaker
# min_duration_sec = 6.0     # minimum duration for OpenVoice input
# target_sr = 22050          # OpenVoice-friendly sample rate
# device = "cuda" if torch.cuda.is_available() else "cpu"
# openvoice_cli_cmd = "python -m openvoice_cli single"  # CLI entry

# # ------------------------
# os.makedirs(speaker_refs_dir, exist_ok=True)
# os.makedirs(cloned_dir, exist_ok=True)
# os.makedirs(padded_dir, exist_ok=True)

# # --------- Helpers ---------
# def load_mono_np(path):
#     wav, sr = torchaudio.load(path)
#     if wav.shape[0] > 1:
#         wav = wav.mean(dim=0, keepdim=True)
#     return wav.squeeze(0).cpu().numpy(), sr

# def resample_np(arr, orig_sr, new_sr):
#     if orig_sr == new_sr:
#         return arr
#     t = torch.from_numpy(arr).unsqueeze(0)
#     resampler = torchaudio.transforms.Resample(orig_sr, new_sr)
#     t2 = resampler(t)
#     return t2.squeeze(0).cpu().numpy()

# # -------- Load diarization ---------
# with open(diarization_json, "r") as f:
#     diar = json.load(f)

# speaker_occ = defaultdict(list)
# for entry in diar:
#     segfile = entry.get("segment_file")
#     for sp in entry.get("speakers", []):
#         speaker_occ[sp["speaker"]].append((segfile, float(sp["start"]), float(sp["end"])))

# print(f"[INFO] Found {len(speaker_occ)} speakers.")

# # -------- Build speaker references ---------
# for speaker, occs in speaker_occ.items():
#     parts = []
#     accum_sec = 0.0
#     for segfile, start, end in occs:
#         seg_path = os.path.join(original_vad_dir, segfile)
#         if not os.path.exists(seg_path):
#             continue
#         arr, sr = load_mono_np(seg_path)
#         s_idx = int(math.floor(start * sr))
#         e_idx = int(math.ceil(end * sr))
#         clip = arr[s_idx:e_idx]
#         if clip.size == 0:
#             continue
#         if sr != target_sr:
#             clip = resample_np(clip, sr, target_sr)
#             sr = target_sr
#         parts.append(clip)
#         accum_sec += clip.shape[0] / float(target_sr)
#         if accum_sec >= ref_seconds:
#             break

#     if len(parts) == 0:
#         print(f"[WARN] No reference for {speaker} — skipping.")
#         continue

#     ref_audio = np.concatenate(parts, axis=0)
#     max_samples = int(ref_seconds * target_sr)
#     if ref_audio.shape[0] > max_samples:
#         ref_audio = ref_audio[:max_samples]
#     elif ref_audio.shape[0] < max_samples:
#         pad = np.zeros(max_samples - ref_audio.shape[0], dtype=ref_audio.dtype)
#         ref_audio = np.concatenate([ref_audio, pad], axis=0)

#     peak = np.max(np.abs(ref_audio)) + 1e-9
#     ref_audio = (ref_audio / peak) * 0.95

#     ref_path = os.path.join(speaker_refs_dir, f"{speaker}.wav")
#     tensor = torch.from_numpy(ref_audio).unsqueeze(0)
#     torchaudio.save(ref_path, tensor, sample_rate=target_sr)
#     print(f"[OK] Saved ref for {speaker} -> {ref_path}")

# print(f"[INFO] Speaker references done at: {speaker_refs_dir}")

# # -------- Primary speaker helper ---------
# def primary_speaker_for_segment(segfile):
#     for entry in diar:
#         if entry.get("segment_file") == segfile:
#             if not entry.get("speakers"):
#                 return None
#             durations = {sp["speaker"]: float(sp["end"])-float(sp["start"]) for sp in entry["speakers"]}
#             return max(durations, key=durations.get)
#     return None

# # --------- Cloning with padding ---------
# translated_files = sorted([f for f in os.listdir(translated_dir) if f.endswith(".wav")])

# for fname in translated_files:
#     in_path_orig = os.path.join(translated_dir, fname)
#     speaker = primary_speaker_for_segment(fname)
#     if speaker is None:
#         print(f"[SKIP] No diarization for {fname}")
#         continue

#     ref_path = os.path.join(speaker_refs_dir, f"{speaker}.wav")
#     if not os.path.exists(ref_path):
#         print(f"[SKIP] Missing reference for speaker {speaker}")
#         continue

#     # Load translated audio
#     audio, sr = load_mono_np(in_path_orig)

#     # Resample if needed
#     if sr != target_sr:
#         audio = resample_np(audio, sr, target_sr)
#         sr = target_sr

#     # Pad short segments to minimum duration
#     curr_duration = len(audio) / sr
#     if curr_duration < min_duration_sec:
#         pad_len = int((min_duration_sec - curr_duration) * sr)
#         audio = np.concatenate([audio, np.zeros(pad_len, dtype=audio.dtype)])
#         print(f"[PAD] {fname} padded from {curr_duration:.2f}s -> {len(audio)/sr:.2f}s")

#     # Save padded audio
#     tmp_path = os.path.join(padded_dir, fname)
#     torchaudio.save(tmp_path, torch.from_numpy(audio).unsqueeze(0), sample_rate=sr)

#     # Run OpenVoice CLI
#     out_path = os.path.join(cloned_dir, fname)
#     cmd = f"{openvoice_cli_cmd} -i {shlex.quote(tmp_path)} -r {shlex.quote(ref_path)} -o {shlex.quote(out_path)} -d {device}"
#     print(f"[RUN] Cloning {fname} -> speaker {speaker}")

#     try:
#         subprocess.run(cmd, shell=True, check=True)
#     except subprocess.CalledProcessError as e:
#         print(f"[ERR] OpenVoice failed for {fname} (err {e.returncode})")
#         continue

# print(f"[INFO] All done. Cloned files are in {cloned_dir}")


In [ ]:
# import os
# from IPython.display import Audio, display

# original_path = "/kaggle/working/OUTPUT/output_audio.wav"
# translated_path = "/kaggle/working/translated_full.wav"

# if os.path.exists(original_path):
#     print("🎵 Original Audio:")
#     display(Audio(filename=original_path))
# else:
#     print(f"⚠️ Original audio not found at {original_path}")

# if os.path.exists(translated_path):
#     print("🎵 Translated Audio:")
#     display(Audio(filename=translated_path))
# else:
#     print(f"⚠️ Translated audio not found at {translated_path}")


In [ ]:
# import os
# import json
# import soundfile as sf
# import numpy as np

# def trim_trailing_silence(audio, target_len, threshold=1e-4):
#     """
#     Trim trailing silence if audio is longer than target_len.
#     Keeps speech intact, only removes quiet tail.
#     """
#     if len(audio) <= target_len:
#         return audio

#     # Find where amplitude falls below threshold near the end
#     energy = np.abs(audio)
#     cutoff = len(audio)
#     while cutoff > target_len and cutoff > 0 and energy[cutoff-1] < threshold:
#         cutoff -= 1

#     # If still longer, hard cut at target_len (failsafe)
#     if cutoff > target_len:
#         cutoff = target_len

#     return audio[:cutoff]


# # --- Paths ---
# timestamps_path = "/kaggle/input/sample-video/EY_VAD/timestamps.json"
# segments_dir = "/kaggle/working/translated"
# merged_audio_path = "/kaggle/working/EY_merged_spa.wav"

# # Load timestamps
# with open(timestamps_path, "r") as f:
#     timestamps = json.load(f)

# # Get reference sample rate
# first_seg_path = os.path.join(segments_dir, "speech_segment_0.wav")
# _, target_sr = sf.read(first_seg_path, dtype="float32")

# # Determine total samples
# total_samples = int(timestamps[-1]["end"])
# merged_audio = np.zeros(total_samples, dtype=np.float32)

# print(f"[INFO] Target SR: {target_sr}, Total length: {total_samples/target_sr:.2f}s")

# # Merge with silence trimming logic
# for idx, seg in enumerate(timestamps):
#     seg_path = os.path.join(segments_dir, f"speech_segment_{idx}.wav")
#     if not os.path.exists(seg_path):
#         print(f"[WARN] Missing {seg_path}, skipping...")
#         continue

#     audio, sr = sf.read(seg_path, dtype="float32")
#     if sr != target_sr:
#         raise ValueError(f"Sample rate mismatch in {seg_path}: {sr} vs {target_sr}")

#     start = int(seg["start"])
#     end = int(seg["end"])
#     slot_len = end - start

#     if len(audio) > slot_len:
#         # Silence-aware trimming
#         audio = trim_trailing_silence(audio, slot_len)
#         if len(audio) < slot_len:
#             # pad again to exact slot length
#             audio = np.pad(audio, (0, slot_len - len(audio)), mode="constant")
#         print(f"[TRIM-SILENCE] {seg_path} reduced to {len(audio)} samples")
#     elif len(audio) < slot_len:
#         # Pad with silence
#         audio = np.pad(audio, (0, slot_len - len(audio)), mode="constant")
#         print(f"[PAD] {seg_path} padded to {slot_len} samples")

#     # Place audio into merged buffer
#     merged_audio[start:start+len(audio)] = audio
#     print(f"[MERGE] {seg_path} placed at {start/target_sr:.2f}s → {end/target_sr:.2f}s")

# # Save final merged file
# sf.write(merged_audio_path, merged_audio, target_sr)
# print(f"[DONE] Final merged audio saved at {merged_audio_path}")


In [ ]:
!pip install ffmpeg-python

In [ ]:
import os

# Paths
input_video = "/kaggle/input/sample-video/input_video.mp4"  # Change to your actual video file path
output_dir = "/kaggle/working/OUTPUT"
os.makedirs(output_dir, exist_ok=True)

output_audio = os.path.join(output_dir, "output_audio.wav")
output_video = os.path.join(output_dir, "output_video.mp4")

# Extract audio as WAV
!ffmpeg -i "{input_video}" -vn -ac 2 -ar 44100 -c:a pcm_s16le "{output_audio}" -y

# Extract video (with original audio or without depending on your need)
# Here, extracting video with original audio intact
!ffmpeg -i "{input_video}" -c copy -an "{output_video}" -y

print("Separation completed!")
print("Audio file:", output_audio)
print("Video file:", output_video)


In [ ]:

# !pip install pyannote.audio

In [ ]:
# from pyannote.audio import Pipeline

# # Load pre-trained speaker diarization pipeline
# pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

# # Path to your audio file
# audio_file = "/kaggle/working/OUTPUT/output_audio.wav"

# # Run diarization
# diarization = pipeline(audio_file)

# # Initialize stats
# speaker_times = {}
# speaker_turns = []
# segments = []

# # Collect segments and stats
# for turn, _, speaker in diarization.itertracks(yield_label=True):
#     start = turn.start
#     end = turn.end
#     duration = end - start
#     speaker_times[speaker] = speaker_times.get(speaker, 0) + duration
#     speaker_turns.append(speaker)
#     segments.append((start, end, speaker))

# # Total meeting time
# meeting_start = min([s[0] for s in segments])
# meeting_end = max([s[1] for s in segments])
# total_meeting_time = meeting_end - meeting_start

# # Number of speakers
# num_speakers = len(speaker_times)

# # Calculate dominance score (% speaking time)
# dominance = {s: (t / total_meeting_time) * 100 for s, t in speaker_times.items()}

# # Count speaker turns
# from collections import Counter
# turn_count = Counter(speaker_turns)

# # Calculate total silence / active time
# segments_sorted = sorted(segments, key=lambda x: x[0])
# silence_time = 0
# for i in range(1, len(segments_sorted)):
#     prev_end = segments_sorted[i-1][1]
#     curr_start = segments_sorted[i][0]
#     gap = curr_start - prev_end
#     if gap > 0:
#         silence_time += gap
# active_time = total_meeting_time - silence_time

# # Print results
# print(f"\nNumber of speakers: {num_speakers}\n")

# print("Total speaking time per speaker:")
# for s, t in speaker_times.items():
#     print(f"{s}: {t:.2f}s")

# print("\nDominance score (% of total meeting time):")
# for s, d in dominance.items():
#     print(f"{s}: {d:.2f}%")

# print("\nSpeaker turn counts:")
# for s, c in turn_count.items():
#     print(f"{s}: {c} turns")

# print(f"\nTotal meeting time: {total_meeting_time:.2f}s")
# print(f"Active speaking time: {active_time:.2f}s")
# print(f"Silence time: {silence_time:.2f}s")


In [ ]:
import json
import os
from pydub import AudioSegment

# ====== INPUT FILES ======
speaker_json = "/kaggle/input/sample-video/EY_VAD/speaker_labels.json"
vad_json = "/kaggle/input/sample-video/EY_VAD/timestamps.json"
translated_dir = "/kaggle/working/translated"
output_file = "/kaggle/working/translated_full1.wav"

# ====== LOAD JSONS ======
with open(speaker_json, "r") as f:
    speaker_data = json.load(f)

with open(vad_json, "r") as f:
    vad_data = json.load(f)

# ====== CONVERT TO MS (assuming 16kHz) ======
for seg in vad_data:
    seg["start"] = int(seg["start"] / 16)   # samples → ms
    seg["end"]   = int(seg["end"] / 16)

# ====== FIND FULL LENGTH ======
original_length = max([seg["end"] for seg in vad_data])
final_audio = AudioSegment.silent(duration=original_length)

# ====== PLACE TRANSLATED SEGMENTS ======
for idx, seg in enumerate(vad_data):
    seg_file = os.path.join(translated_dir, f"speech_segment_{idx}.wav")

    if not os.path.exists(seg_file):
        print(f"⚠️ Missing file: {seg_file}, skipping...")
        continue

    translated = AudioSegment.from_wav(seg_file)

    # place at converted start_ms
    start_ms = seg["start"]
    final_audio = final_audio.overlay(translated, position=start_ms)

# ====== EXPORT ======
final_audio.export(output_file, format="wav")
print(f"✅ Final translated audio saved as {output_file}, length {len(final_audio)/1000:.2f} sec")


In [ ]:
import os
from IPython.display import Audio, display

def play_audios_in_path(path):
    if not os.path.exists(path):
        print(f"⚠️ Path not found: {path}")
        return
    
    if os.path.isfile(path):
        # If a single file is given
        if path.lower().endswith((".wav", ".mp3", ".flac", ".ogg")):
            print(f"🎵 Playing: {os.path.basename(path)}")
            display(Audio(filename=path))
        else:
            print(f"⚠️ Not an audio file: {path}")
        return
    
    # If a directory is given
    files = [f for f in os.listdir(path) if f.lower().endswith((".wav", ".mp3", ".flac", ".ogg"))]
    if not files:
        print(f"⚠️ No audio files found in {path}")
        return
    
    for f in files:
        file_path = os.path.join(path, f)
        print(f"🎵 Playing: {f}")
        display(Audio(filename=file_path))

# Example usage:
play_audios_in_path("/kaggle/working/OUTPUT/output_audio.wav")
play_audios_in_path("/kaggle/working/translated_full1.wav")
